In [2]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split

from datetime import datetime
import sys
sys.path.insert(1, '/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/util_funcs/')
from pandas_colFuncs import whichBisSig_NAHS, B_ID, ancestor_below_B

In [3]:
now = datetime.now()
print("time at start =", now)

time at start = 2021-10-25 09:36:11.617448


In [4]:
save_data = True
save_preprocessedDataframe = True
tmp_data = False
take_subset = False
subset_size = 100000

delete_top_numFSP_evts=True

In [5]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/"

data_subdir = "NAHSA_creation1/"
root_subdir = "NAHS_100kRun1/"  

root_path = nfs_path + "NAHS/onlineDataProd/" + root_subdir 
#root_path = "/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/"

In [6]:
merged = "merged_"
if tmp_data:
    merged += "tmp_"
merged = ""

In [8]:
fileHc = uproot.open(root_path + merged + "allHc.root:variables")

In [9]:
df_Hc = fileHc.arrays(library="pd")

In [10]:
names = ["gammas","electrons","pions","kaons","muons"]
dfs = []
for name in names:
    filename=root_path + merged + "{}.root:variables".format(name)
    print(filename)
    tmpFileFSPs = uproot.open(filename)
    df_tmp = tmpFileFSPs.arrays(library="pd")
    dfs.append(df_tmp)

/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_100kRun1/gammas.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_100kRun1/electrons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_100kRun1/pions.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_100kRun1/kaons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_100kRun1/muons.root:variables


In [11]:
df_FSPs = pd.concat(dfs)

In [14]:
# delete FSPs for which no Y4S file entry was found
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [15]:
df_FSPs.keys()

Index(['__experiment__', '__run__', '__event__', '__candidate__',
       '__ncandidates__', '__weight__', 'genMothPDG_0', 'genMothPDG_1',
       'genMothPDG_2', 'genMothPDG_3', 'genMothPDG_4', 'genMothPDG_5',
       'genMothPDG_6', 'genMothPDG_7', 'genMothPDG_8', 'genMothPDG_9',
       'genMotherID_0', 'genMotherID_1', 'genMotherID_2', 'genMotherID_3',
       'genMotherID_4', 'genMotherID_5', 'genMotherID_6', 'genMotherID_7',
       'genMotherID_8', 'genMotherID_9', 'mcMother0_uniqParID',
       'mcMother1_uniqParID', 'mcMother2_uniqParID', 'mcMother3_uniqParID',
       'mcMother4_uniqParID', 'mcMother5_uniqParID', 'mcMother6_uniqParID',
       'mcMother7_uniqParID', 'mcMother8_uniqParID', 'mcMother9_uniqParID',
       'cmpx', 'cmpy', 'cmpz', 'cmE', 'isSignal', 'uniqueParticleIdentifier',
       'mcErrors', 'mcPDG', 'genMotherID', 'genMotherP', 'genMotherPDG',
       'charge', 'dr', 'dz', 'clusterReg', 'clusterE9E21', 'M', 'PDG',
       'genParticleID', 'px', 'py', 'pz', 'pt', 'p', 'E'

In [30]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])
print(df_Hc["__event__"].nunique())

14271943
856354
808996


## check that each particles occurs only ones

In [17]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()
groupsFSPs_uniqParID.value_counts('count')

count
1    14271943
dtype: int64

## add cols

In [18]:
df_Hc['Bsig_uniqParID'] = df_Hc.apply(whichBisSig_NAHS, axis=1)

In [19]:
df_Hc['Bsig_uniqParID'].value_counts()

83886081.0    449802
83886082.0    406552
Name: Bsig_uniqParID, dtype: int64

In [20]:
# function to create col with the particles mother B's uniqueParticleIdentifier
df_FSPs['B_ID'] = df_FSPs.apply(B_ID, axis=1)

In [21]:
df_FSPs['B_ID'].value_counts()

83886081    6184364
83886082    6183529
0           1904050
Name: B_ID, dtype: int64

In [22]:
df_FSPs['PDG'].value_counts()

 22.0     7511805
 211.0    1767462
-211.0    1674598
 13.0      728741
-13.0      714377
 321.0     507578
-321.0     502421
-11.0      436685
 11.0      428276
Name: PDG, dtype: int64

In [26]:
df_Hc['PDG'].value_counts()

 413.0     277655
-413.0     270274
 411.0     104340
-411.0     103734
 421.0      39840
-421.0      26120
-431.0       9194
 431.0       9143
 423.0       5280
 443.0       4105
-423.0       3504
-4122.0      1606
 4122.0      1559
Name: PDG, dtype: int64

## double between gamma and charged?

In [23]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()
groupsFSPs_uniqParID["count"].max()

1

## check D*lnu

In [24]:
groupsHc = pd.DataFrame({'count' : df_Hc[df_Hc["Bsig_uniqParID"] == 83886082.0].groupby( 
    ["genUp4S_PDG_1_0","genUp4S_PDG_1_1","genUp4S_PDG_1_2"] ).size()}).reset_index()

groupsHc

,genUp4S_PDG_1_0,genUp4S_PDG_1_1,genUp4S_PDG_1_2,count
0,-413.0,-13.0,14.0,100891
1,-413.0,-11.0,12.0,101200
2,413.0,11.0,-12.0,101511
3,413.0,13.0,-14.0,102950


In [25]:
groupsHc = pd.DataFrame({'count' : df_Hc[df_Hc["Bsig_uniqParID"] == 83886081.0].groupby( 
    ["genUp4S_PDG_0_0","genUp4S_PDG_0_1","genUp4S_PDG_0_2"] ).size()}).reset_index()

groupsHc

,genUp4S_PDG_0_0,genUp4S_PDG_0_1,genUp4S_PDG_0_2,count
0,-413.0,-13.0,14.0,111349
1,-413.0,-11.0,12.0,112250
2,413.0,11.0,-12.0,113530
3,413.0,13.0,-14.0,112673


## delete evts where Hc mother and Bsig overlap

In [29]:
uniqParID = pd.DataFrame({'count' : df_Hc.groupby( ["__event__"] ).size()}).reset_index()
uniqParID["count"].max()

6

In [27]:
print(df_Hc.shape[0])

856354


In [31]:
df_Hc[df_Hc['Bsig_uniqParID'] == df_Hc['mcMother0_uniqParID']].shape[0]
#[["__event__","uniqueParticleIdentifier","PDG","genMothPDG_0",'Bsig_uniqParID',"mcMother0_uniqParID"]][:10]

402992

In [32]:
df_Hc = df_Hc[df_Hc['Bsig_uniqParID'] != df_Hc['mcMother0_uniqParID']]

In [33]:
print(df_Hc.shape[0])

453362


In [76]:
#[["__event__","uniqueParticleIdentifier","PDG","genMothPDG_0",'Bsig_uniqParID',"mcMother0_uniqParID"]]

## take care of duplis in df_Hc, delete them, because with the prior, duplis here mean there must have been an error

In [34]:
groupsHc_uniqParID = pd.DataFrame({'count' : df_Hc.groupby( ["__event__"] ).size()}).reset_index()
groupsHc_uniqParID.value_counts('count')

count
1    437440
2      7706
3       152
4        12
6         1
dtype: int64

In [35]:
df_Hc[df_Hc['__event__'].isin(groupsHc_uniqParID[groupsHc_uniqParID["count"]>1]["__event__"])][["__event__",
    "uniqueParticleIdentifier","PDG",
    "genMothPDG_0",'Bsig_uniqParID',"mcMother0_uniqParID","px","E","x","genUp4S_PDG_1_0"]][:10]

,__event__,uniqueParticleIdentifier,PDG,genMothPDG_0,Bsig_uniqParID,mcMother0_uniqParID,px,E,x,genUp4S_PDG_1_0
50,3679955,100663296.0,-421.0,511.0,83886081.0,83886082.0,-0.079056,1.939253,-0.042812,411.0
56,243575,100663296.0,-421.0,-511.0,83886081.0,83886082.0,0.325505,2.003441,-0.044740,-421.0
77,9744,100663296.0,-421.0,-511.0,83886081.0,83886082.0,0.841873,2.300964,-0.052184,411.0
94,14108,100663296.0,421.0,-511.0,83886081.0,83886082.0,-0.058585,1.946033,-0.046022,-321.0
95,14108,100663296.0,421.0,-511.0,83886081.0,83886082.0,0.619852,2.303867,-0.056158,-321.0
117,4558,100663296.0,421.0,-511.0,83886082.0,83886081.0,-0.296814,1.911936,-0.561030,-413.0
118,4558,100663296.0,421.0,-511.0,83886082.0,83886081.0,-0.300684,1.894096,-0.043345,-413.0
121,5600,100663296.0,-421.0,511.0,83886082.0,83886081.0,0.179031,2.151420,-0.047189,413.0
122,5600,100663296.0,421.0,511.0,83886082.0,83886081.0,-0.384867,2.601196,0.000000,413.0
124,5661,100663296.0,-421.0,-511.0,83886082.0,83886081.0,0.078100,1.885611,-0.044987,-413.0


In [36]:
df_Hc = df_Hc[df_Hc['__event__'].isin(groupsHc_uniqParID[groupsHc_uniqParID["count"]==1]["__event__"])]

In [37]:
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [38]:
groupsHc_uniqParID = pd.DataFrame({'count' : df_Hc.groupby( ["__event__"] ).size()}).reset_index()
groupsHc_uniqParID.value_counts('count')

count
1    437440
dtype: int64

## delete the H_c FSPs 

In [39]:
df_FSPs['ancestor_below_B_arrayID'] = df_FSPs.apply(ancestor_below_B, axis=1)

In [40]:
df_Hc["HcMother_uniqParID"] = df_Hc["mcMother0_uniqParID"]
df_Hc["Hc_genParticleID"] = df_Hc["genParticleID"]
df_Hc["Hc_MotherPDG"] = df_Hc["genMothPDG_0"]

In [41]:
df_FSPs = pd.merge(df_FSPs,df_Hc[["__event__","Bsig_uniqParID","HcMother_uniqParID","Hc_genParticleID","Hc_MotherPDG"]], on="__event__")

In [42]:
df_FSPs[["__event__","Bsig_uniqParID","HcMother_uniqParID","Hc_genParticleID",
         "ancestor_below_B_arrayID","Hc_MotherPDG"]][:10]

,__event__,Bsig_uniqParID,HcMother_uniqParID,Hc_genParticleID,ancestor_below_B_arrayID,Hc_MotherPDG
0,2331192,83886082.0,83886081.0,3.0,0.0,511.0
1,2331192,83886082.0,83886081.0,3.0,0.0,511.0
2,2331192,83886082.0,83886081.0,3.0,6.0,511.0
3,2331192,83886082.0,83886081.0,3.0,4.0,511.0
4,2331192,83886082.0,83886081.0,3.0,6.0,511.0
5,2331192,83886082.0,83886081.0,3.0,6.0,511.0
6,2331192,83886082.0,83886081.0,3.0,0.0,511.0
7,2331192,83886082.0,83886081.0,3.0,4.0,511.0
8,2331192,83886082.0,83886081.0,3.0,6.0,511.0
9,2331192,83886082.0,83886081.0,3.0,3.0,511.0


In [43]:
df_FSPs.shape[0]

7745336

In [44]:
# deletion of Hc FSPs, those who 
df_FSPs = df_FSPs[~((df_FSPs["Hc_genParticleID"] == df_FSPs["ancestor_below_B_arrayID"]) &  # originate from Hc arrayID(genParticleID)
                  (df_FSPs["HcMother_uniqParID"] == df_FSPs["B_ID"]))] # and are B-tag ancestors

In [45]:
df_FSPs.shape[0]

5892803

## delete delete_top_numFSP_evts

In [47]:
numberFSPsEvts = pd.DataFrame({'count' : df_FSPs.groupby(["__event__"]).size()}).reset_index()
numEvtsBeforeCut = df_Hc.shape[0]

In [48]:
# print first to adjust percentile used
print("original max FSPs:",numberFSPsEvts["count"].max())
for percentile_lvl in [0.995,0.996,0.997,0.998,0.999]:
    print(percentile_lvl)
    percentile = numberFSPsEvts["count"].quantile(q=percentile_lvl)
    cut_numberFSPsEvts = numberFSPsEvts[numberFSPsEvts["count"] < percentile]
    
    #TMP_df_FSPs = df_FSPs[df_FSPs['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    TMP_df_Hc = df_Hc[df_Hc['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    
    print("would delete", round(100 - (TMP_df_Hc.shape[0]/df_Hc.shape[0])*100,2),"% of evts")
    print("new max FSPs:",cut_numberFSPsEvts["count"].max())

original max FSPs: 34
0.995
would delete 0.81 % of evts
new max FSPs: 23
0.996
would delete 0.43 % of evts
new max FSPs: 24
0.997
would delete 0.43 % of evts
new max FSPs: 24
0.998
would delete 0.23 % of evts
new max FSPs: 25
0.999
would delete 0.11 % of evts
new max FSPs: 26


In [50]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])


if delete_top_numFSP_evts:
    delete_percentileLvl = 0.998
    print("used delete_percentileLvl:",delete_percentileLvl)
    
    
    print(numberFSPsEvts["count"].describe())
    
    percentile = numberFSPsEvts["count"].quantile(q=delete_percentileLvl)
    cut_numberFSPsEvts = numberFSPsEvts[numberFSPsEvts["count"] < percentile]
    
    df_FSPs = df_FSPs[df_FSPs['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    df_Hc = df_Hc[df_Hc['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    
print(df_FSPs.shape[0])
print(df_Hc.shape[0])
print("deleted", round(100 - (df_Hc.shape[0]/numEvtsBeforeCut),2)*100,"% of events")
numberFSPsEvts = pd.DataFrame({'count' : df_FSPs.groupby(["__event__"]).size()}).reset_index()
print(numberFSPsEvts["count"].describe())

5892803
437440
used delete_percentileLvl: 0.998
count    437440.000000
mean         13.471111
std           3.961210
min           1.000000
25%          11.000000
50%          13.000000
75%          16.000000
max          34.000000
Name: count, dtype: float64
5866197
436451
deleted 100 % of events
count    436451.000000
mean         13.440677
std           3.913238
min           1.000000
25%          11.000000
50%          13.000000
75%          16.000000
max          25.000000
Name: count, dtype: float64


## impute the nan values of variables

In [52]:
## data preprocessing, dealing with NaN
imputelist = [["pionID",-1.],["kaonID",-1.],["electronID",-1.],["muonID",-1.],["protonID",-1.]
              ,["clusterReg",-1.],["clusterE9E21",-1.]
              ]
for impute in imputelist:
    column_name = impute[0]
    impute_val = impute[1]
    mask = df_FSPs[column_name].isna() == 1
    df_FSPs.loc[mask, column_name] = impute_val
    print(df_FSPs[df_FSPs[column_name].isna() == True].shape[0])

0
0
0
0
0
0
0


## last check if events match between dfs

In [53]:
df_Hc = df_Hc[df_Hc['__event__'].isin(df_FSPs["__event__"])]
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [54]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])

5866197
436451


## save the dfs as they will be used for data prod

In [55]:
df_FSPs_final = df_FSPs
df_Y4S_final = df_Hc

In [18]:
root_path

'/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_100kRun1/'

In [57]:
df_FSPs_final.to_csv(root_path + "final_df_FSPs_preProcessed_NAHS.csv")
df_Y4S_final.to_csv(root_path + "final_df_Y4S_preProcessed_NAHS.csv")

## load the df's

In [6]:
df_FSPs = pd.read_csv(root_path + "final_df_FSPs_preProcessed_NAHS.csv".format(subset_size))
df_Y4S_final = pd.read_csv(root_path + "final_df_Y4S_preProcessed_NAHS.csv")

## check the subset of (B-sig side) D*-> D0 (-> kpi, kpipi,...) pi  

In [7]:
df_Y4S_final.keys()

Index(['Unnamed: 0', '__experiment__', '__run__', '__event__', '__candidate__',
       '__ncandidates__', '__weight__', 'extraInfo__boSignalProbability__bc',
       'isSignalAcceptMissingGamma', 'PDG', 'genUp4S_uniqParID_0',
       'genUp4S_PDG_0_0', 'abs_genUp4S_PDG_0_0', 'genUp4S_uniqParID_0_0',
       'genUp4S_PDG_0_0_0', 'genUp4S_uniqParID_0_0_0', 'genUp4S_PDG_0_0_0_0',
       'genUp4S_uniqParID_0_0_0_0', 'genUp4S_PDG_0_0_0_1',
       'genUp4S_uniqParID_0_0_0_1', 'genUp4S_PDG_0_0_0_2',
       'genUp4S_uniqParID_0_0_0_2', 'genUp4S_PDG_0_0_0_3',
       'genUp4S_uniqParID_0_0_0_3', 'genUp4S_PDG_0_0_0_4',
       'genUp4S_uniqParID_0_0_0_4', 'genUp4S_PDG_0_0_1',
       'genUp4S_uniqParID_0_0_1', 'genUp4S_PDG_0_1', 'abs_genUp4S_PDG_0_1',
       'genUp4S_uniqParID_0_1', 'genUp4S_PDG_0_2', 'abs_genUp4S_PDG_0_2',
       'genUp4S_uniqParID_0_2', 'genUp4S_uniqParID_1', 'genUp4S_PDG_1_0',
       'abs_genUp4S_PDG_1_0', 'genUp4S_uniqParID_1_0', 'genUp4S_PDG_1_0_0',
       'genUp4S_uniqParID_1_0_

In [8]:
df_Y4S_final["Bsig_uniqParID"].value_counts()

83886081.0    228438
83886082.0    208013
Name: Bsig_uniqParID, dtype: int64

In [17]:
df_Y4S_final[(df_Y4S_final["Bsig_uniqParID"] == 83886081.0)  &
            (df_Y4S_final["genUp4S_PDG_0_0_0"].abs() == 421.0)  & # 421=D0
            (df_Y4S_final["genUp4S_PDG_0_0_1"].abs() == 211.0)  & # 211=pion
            (df_Y4S_final["genUp4S_PDG_0_0_0_0"].abs() == 321.0)  & # 321=kaon 
            (df_Y4S_final["genUp4S_PDG_0_0_0_1"].abs() == 211.0)  & # 211=pion
            (df_Y4S_final["genUp4S_PDG_0_0_0_2"].abs() == 211.0)  | # 211=pion
            (df_Y4S_final["genUp4S_PDG_0_0_0_2"].abs() == 111.0)
            |
            (df_Y4S_final["Bsig_uniqParID"] == 83886082.0)  &
            (df_Y4S_final["genUp4S_PDG_1_0_0"].abs() == 421.0)  & # 421=D0
            (df_Y4S_final["genUp4S_PDG_1_0_1"].abs() == 211.0)  & # 211=pion
            (df_Y4S_final["genUp4S_PDG_1_0_0_0"].abs() == 321.0)  & # 321=kaon 
            (df_Y4S_final["genUp4S_PDG_1_0_0_1"].abs() == 211.0)  & # 211=pion
            (df_Y4S_final["genUp4S_PDG_1_0_0_2"].abs() == 211.0)  | # 211=pion
            (df_Y4S_final["genUp4S_PDG_1_0_0_2"].abs() == 111.0)] # 211=pion

,Unnamed: 0,__experiment__,__run__,__event__,__candidate__,__ncandidates__,__weight__,extraInfo__boSignalProbability__bc,isSignalAcceptMissingGamma,PDG,...,E,x,y,z,uniqueParticleIdentifier,genParticleID,Bsig_uniqParID,HcMother_uniqParID,Hc_genParticleID,Hc_MotherPDG
1,1,1003,0,2331997,0,1,1.0,0.320647,1.0,-421.0,...,2.474679,-0.036619,0.012706,0.040069,100663296.0,7.0,83886081.0,83886082.0,7.0,-511.0
3,3,1003,0,2332405,0,1,1.0,0.932970,1.0,421.0,...,2.807157,-0.060034,-0.025120,-0.020865,100663296.0,5.0,83886082.0,83886081.0,5.0,511.0
6,6,1003,0,2334251,0,1,1.0,0.366874,1.0,-421.0,...,1.966530,-0.047907,0.009682,0.009759,100663296.0,4.0,83886082.0,83886081.0,4.0,-511.0
7,7,1003,0,2335227,0,1,1.0,0.364355,1.0,-421.0,...,2.267325,-0.050209,0.029724,0.017411,100663296.0,7.0,83886081.0,83886082.0,7.0,511.0
9,9,1003,0,2335364,0,1,1.0,0.307449,1.0,-421.0,...,2.166376,-0.066308,0.014497,0.016507,100663296.0,3.0,83886082.0,83886081.0,3.0,511.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436411,856304,1003,0,50543887,0,1,1.0,0.367751,1.0,-4122.0,...,2.352582,-0.054613,0.014061,-0.002140,100663296.0,9.0,83886081.0,83886082.0,9.0,511.0
436421,856314,1003,0,45948062,0,1,1.0,0.033152,1.0,-4122.0,...,2.413146,-0.051283,0.018652,-0.009662,100663296.0,7.0,83886081.0,83886082.0,7.0,-511.0
436433,856326,1003,0,37549717,0,1,1.0,0.013528,1.0,4122.0,...,2.753273,-0.043266,0.021590,0.002805,100663296.0,5.0,83886082.0,83886081.0,5.0,-511.0
436441,856339,1003,0,34128104,0,1,1.0,0.134273,1.0,-4122.0,...,2.446933,-0.050287,0.020066,-0.026647,100663296.0,3.0,83886082.0,83886081.0,3.0,-511.0


In [ ]:
D0:kpi 
D0:kpipipi 
D0:kpipi0     
D0:kpipipipi0 

## beginning of data prod

In [58]:
numFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["__event__"] ).size()}).reset_index()

minFSPs = numFSPs["count"].min()
maxFSPs = numFSPs["count"].max()
print("minFSPs:",minFSPs)
print("maxFSPs:",maxFSPs,'\n')
print("num Events:",numFSPs.shape[0],'\n')

df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')

minFSPs: 1
maxFSPs: 25 

num Events: 436451 



In [69]:
variable_combinations = [
    [[    "charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_M"],
    [["M",         "dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_charge"],
    [["M","charge",     "dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_dr"],
    [["M","charge","dr",     "clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_dz"],
    [["M","charge","dr","dz",             "clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_clReg"],
    [["M","charge","dr","dz","clusterReg",               "pionID","kaonID","electronID","muonID","protonID"],"NAHSA_no_E9E21"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21",         "kaonID","electronID","muonID","protonID"],"NAHSA_no_pID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID",         "electronID","muonID","protonID"],"NAHSA_no_kID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID",             "muonID","protonID"],"NAHSA_no_eID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID"         ,"protonID"],"NAHSA_no_muID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID"           ],"NAHSA_no_proID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"],"NAHSA_allExtras"],
    [["M","charge","pionID","kaonID","electronID","muonID"],"NAHSA_IDchargeM"],
    [["M","charge"],"NAHSA_chargeM"],
    [["charge","pionID","kaonID","electronID","muonID"],"NAHSA_chargeIDs"],
    [["charge","pionID"],"NAHSA_chargePionID"],
    #[[],"NAHSA_only4mom"],
    [["charge"],"NAHSA_charge"]
]

In [65]:
variable_combinations = [
    [[],"NAHSA_only4mom"]
]

In [66]:
minFSPs = 5

In [ ]:
for var_comb in variable_combinations:
    ablationSub = var_comb[1]
    data_dir = Path(nfs_path + "data/" + data_subdir + root_subdir + ablationSub + "/")    
    if save_data:
        data_dir.mkdir(parents=True, exist_ok=True)
    print("Will save data to:", data_dir,'is', save_data ,'\n')


    
    for num_FSPs_toData in range(minFSPs, maxFSPs+1):
        df_num_subset = df_FSPs_final.copy()
        df_num_subset = df_num_subset[df_num_subset['numFSPs'] == num_FSPs_toData]


        numEvents = df_num_subset.__event__.nunique()
        print("numEvents:",numEvents)
        print("num_FSPs_toData:",num_FSPs_toData)  
        if numEvents == 0:
            print("skipped because empty \n")
            continue

        if numEvents < 10:
            print("skipped because <10 events \n")
            continue

        num_features = 4 + len(var_comb[0])
        leaves = np.zeros((numEvents, num_FSPs_toData,  num_features))  
        SA_target =  np.zeros((numEvents, num_FSPs_toData))
        global_tag = np.chararray((numEvents, num_FSPs_toData + 1), itemsize=30)

        event_list = df_num_subset[df_num_subset["numFSPs"] == num_FSPs_toData]["__event__"].unique()
        #print("len(event_list):",len(event_list))
        for i in range(numEvents):

            event_iter = event_list[i]

            global_tag_masterInfo = "evt" + str(event_iter)
            global_tag[i,-1] = global_tag_masterInfo
            #print("global_tag[i,-1]:",global_tag[i,-1])
            #print("i:",i,"event_iter:",event_iter)

            event_df = df_num_subset[df_num_subset.__event__ == event_iter]

            for j in range(num_FSPs_toData):
                #print("numParticle:",j)
                particle = event_df.iloc[j]

                #print(particle["mcPDG"],particle["px"],particle["py"],particle["pz"],particle["E"])
                leaves[i,j,0] = particle["px"]
                leaves[i,j,1] = particle["py"]
                leaves[i,j,2] = particle["pz"]
                leaves[i,j,3] = particle["E"]
                
                for k in range(len(var_comb[0])):
                    leav_idx = k + 4
                    leaves[i,j,leav_idx] = particle[var_comb[0][k]]
                    



                global_tag_Info = str((particle["mcPDG"])) 
                global_tag[i,j] = global_tag_Info

                label = -10 # error code if assignment fails
                B_tag_uniqID = -10 # error code if assignment fails     

                B_sig_uniqID = df_Y4S_final[df_Y4S_final["__event__"] == event_iter].iloc[0]['Bsig_uniqParID']
                if B_sig_uniqID == 83886082.0:
                    B_tag_uniqID = 83886081.0
                elif B_sig_uniqID == 83886081.0:
                    B_tag_uniqID = 83886082.0

                if particle["B_ID"] == B_tag_uniqID:
                    label = 1 # particle belongs to X (MC truth)
                elif particle["B_ID"] == B_sig_uniqID:
                    label = 2 # particle belongs to Bsig (MC truth)
                elif particle["B_ID"] == 0:
                    label = 0 # background (MC truth)


                SA_target[i,j] = label

            del event_df


        # shuffle the data    
        for idx in np.arange(leaves.shape[0]):   # arange is like range but gives ndarray instead of list
            perms = np.random.permutation(leaves.shape[1])

            leaves[idx,:] = leaves[idx,perms]
            SA_target[idx,:] = SA_target[idx,perms]
            global_tag[idx,0:-1] = global_tag[idx,perms]





        #print(global_tag)
        train_ratio = 0.7
        validation_ratio = 0.2
        test_ratio = 0.1

        print("leaves.shape:",leaves.shape)
        print("SA_target.shape:",SA_target.shape)
        print("global_tag.shape:",global_tag.shape)


        print("leaves[0]:",leaves[0])
        print("SA_target[0]:",SA_target[0])
        print("global_tag[0]:",global_tag[0])

        x=leaves
        y=SA_target
        z=global_tag

        #x_train, x_val, y_train, y_val, z_train, z_val = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
        x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
        x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(x_test, y_test, z_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=False) 

        if save_data==True:
            np.save(data_dir / "leaves_train_FSP{}.npy".format(num_FSPs_toData), x_train)
            np.save(data_dir / "is_left_arr_train_FSP{}.npy".format(num_FSPs_toData), y_train)
            np.save(data_dir / "global_tag_train_FSP{}.npy".format(num_FSPs_toData), z_train)

            np.save(data_dir / "leaves_val_FSP{}.npy".format(num_FSPs_toData), x_val)
            np.save(data_dir / "is_left_arr_val_FSP{}.npy".format(num_FSPs_toData), y_val)
            np.save(data_dir / "global_tag_val_FSP{}.npy".format(num_FSPs_toData), z_val)

            np.save(data_dir / "leaves_test_FSP{}.npy".format(num_FSPs_toData), x_test)
            np.save(data_dir / "is_left_arr_test_FSP{}.npy".format(num_FSPs_toData), y_test)
            np.save(data_dir / "global_tag_test_FSP{}.npy".format(num_FSPs_toData), z_test)


        print("")


        del df_num_subset


Will save data to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/data/NAHSA_creation1/NAHS_100kRun1/NAHSA_no_M is True 

numEvents: 3591
num_FSPs_toData: 5
leaves.shape: (3591, 5, 14)
SA_target.shape: (3591, 5)
global_tag.shape: (3591, 6)
leaves[0]: [[-8.14568579e-01  1.74185324e+00  1.08853984e+00  2.20963660e+00
  -1.00000000e+00  1.54155116e-03  5.47560336e-02  2.00000000e+00
   9.81445312e-01  2.66938441e-11  2.37415073e-14  1.00000000e+00
   4.99273016e-22  2.12655895e-15]
 [-2.38767564e-01 -6.05019033e-01  3.56785566e-01  7.54873375e-01
   1.00000000e+00  6.56575131e-03  4.89720344e-02  2.00000000e+00
   7.87109375e-01  9.12002192e-01  4.85960980e-11  1.57489225e-06
   8.79962329e-02  2.05496368e-21]
 [ 8.10032308e-01 -5.51182985e-01  3.75340253e-01  1.04920700e+00
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   9.95117188e-01 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [ 3.02234460e-02  5.00667244e-02  2.


numEvents: 32206
num_FSPs_toData: 10
leaves.shape: (32206, 10, 14)
SA_target.shape: (32206, 10)
global_tag.shape: (32206, 11)
leaves[0]: [[ 9.80079472e-02 -4.69597578e-02  8.80707428e-02  1.39882924e-01
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   9.86328125e-01 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [-4.21528593e-02  2.65963525e-02  1.61919650e-02  5.24061947e-02
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   9.61914062e-01 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [-8.59866366e-02  6.77343607e-02  4.77126352e-02  1.19407457e-01
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   9.89257812e-01 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [-3.31587419e-02  2.96551675e-01  1.54830098e-01  3.36176676e-01
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   9.89257812e-01 -1.00000000e+


numEvents: 42819
num_FSPs_toData: 13
leaves.shape: (42819, 13, 14)
SA_target.shape: (42819, 13)
global_tag.shape: (42819, 14)
leaves[0]: [[-8.93043634e-03  1.80949971e-01  3.79183190e-03  1.81209886e-01
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [ 1.54823318e-01 -2.07968211e+00  1.21251547e+00  2.41462324e+00
   1.00000000e+00  3.55311317e-04  6.10923159e-02  2.00000000e+00
   1.00000000e+00  5.05497945e-13  1.31196676e-13  7.61975650e-64
   1.00000000e+00  4.05001999e-26]
 [-2.27028668e-01 -7.33320042e-02  1.02306016e-01  2.94730406e-01
   1.00000000e+00  1.22099505e-03  7.44024439e-02 -1.00000000e+00
  -1.00000000e+00  3.45872358e-01  2.56999686e-20  6.02067700e-01
   5.20599419e-02  4.79997612e-35]
 [ 1.33282864e+00 -1.09681457e-01  1.94088185e+00  2.35700760e+00
  -1.00000000e+00  5.74448600e-03  6.52670762e-02  2.00000000e+00
   9.83398438e-01  6.22258406e-


numEvents: 34253
num_FSPs_toData: 16
leaves.shape: (34253, 16, 14)
SA_target.shape: (34253, 16)
global_tag.shape: (34253, 17)
leaves[0]: [[-4.55206148e-02  1.57436728e-01  1.55181482e-01  2.25698344e-01
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   9.95117188e-01 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [-1.07367843e-01 -2.19028052e-02 -1.12766400e-02  1.10157838e-01
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [-4.12536897e-02 -7.40044340e-02 -6.12548133e-03  1.63388914e-01
   1.00000000e+00  1.07600279e+00 -4.08143110e-01 -1.00000000e+00
  -1.00000000e+00  9.99922417e-01  3.29677214e-05  1.36793613e-84
   3.73362374e-05  4.68196153e-06]
 [ 1.14713743e-01 -1.67829767e-02  3.06301922e-01  3.27508440e-01
   0.00000000e+00  1.18686603e-09  2.23517418e-10  1.00000000e+00
   9.88281250e-01 -1.00000000e+


numEvents: 22255
num_FSPs_toData: 18
leaves.shape: (22255, 18, 14)
SA_target.shape: (22255, 18)
global_tag.shape: (22255, 19)
leaves[0]: [[-1.81560263e-01  4.77777064e-01  5.14179647e-01  8.77116145e-01
   1.00000000e+00  2.43826925e-03  6.14079201e-02  2.00000000e+00
   7.11914062e-01  5.10221333e-07  9.86885264e-01  1.31141522e-02
   7.34684701e-08  1.66929388e-64]
 [-7.29059130e-02 -1.50300771e-01  2.79857635e-01  3.25923135e-01
   0.00000000e+00  1.18686603e-09  2.23517418e-10  1.00000000e+00
   1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [-9.90705013e-01  1.73360392e-01  2.13107681e+00  2.35648868e+00
   1.00000000e+00  2.45847283e-03  9.86638789e-02  1.00000000e+00
   9.72656250e-01  4.32081387e-05  4.27802948e-25  9.99956792e-01
   3.45210413e-11  3.31546451e-26]
 [-9.60487649e-02  3.24819058e-01 -3.51819843e-02  3.40544502e-01
   0.00000000e+00  1.18686603e-09  2.23517418e-10  2.00000000e+00
   1.00000000e+00 -1.00000000e+

...


numEvents: 1658
num_FSPs_toData: 24
leaves.shape: (1658, 24, 15)
SA_target.shape: (1658, 24)
global_tag.shape: (1658, 25)
leaves[0]: [[-1.11181609e-01  2.04207450e-01  1.43541843e-01  2.73251338e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  9.82421875e-01 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [ 1.85045034e-01 -3.68747205e-01  3.33009601e-01  5.48262211e-01
   1.39570385e-01 -1.00000000e+00  2.28231761e-03 -3.52705036e-02
   2.00000000e+00  9.35546875e-01  7.31186938e-01  3.96979064e-05
   3.88747111e-02  2.29898653e-01  7.82849452e-28]
 [ 4.96155769e-02 -1.22058906e-01  1.78297281e-01  2.21697998e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  9.33593750e-01 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [ 9.56399366e-02  3.30931321e-02 -1.89754125e-02  1.02967078e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.


Will save data to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/data/NAHSA_creation1/NAHS_100kRun1/NAHSA_IDchargeM is True 

numEvents: 3591
num_FSPs_toData: 5
leaves.shape: (3591, 5, 10)
SA_target.shape: (3591, 5)
global_tag.shape: (3591, 6)
leaves[0]: [[ 3.94925773e-02 -6.26280755e-02  7.02249110e-02  1.72896993e-01
   1.39570385e-01  1.00000000e+00  9.99952085e-01  5.10187786e-08
   8.42826485e-22  4.78619153e-05]
 [-2.38767564e-01 -6.05019033e-01  3.56785566e-01  7.54873375e-01
   1.39570385e-01  1.00000000e+00  9.12002192e-01  4.85960980e-11
   1.57489225e-06  8.79962329e-02]
 [ 3.02234460e-02  5.00667244e-02  2.19515767e-02  6.24660332e-02
   0.00000000e+00  0.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [ 8.10032308e-01 -5.51182985e-01  3.75340253e-01  1.04920700e+00
   0.00000000e+00  0.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [-8.14568579e-01  1.74185324e+00  1.08853984e+00  2.20963660e


numEvents: 37398
num_FSPs_toData: 11
leaves.shape: (37398, 11, 10)
SA_target.shape: (37398, 11)
global_tag.shape: (37398, 12)
leaves[0]: [[ 1.60634637e-01 -1.85178057e-03  1.41748667e-01  2.14241920e-01
   0.00000000e+00  0.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [ 4.91621213e-05 -2.15924725e-01  3.83435130e-01  4.40052256e-01
   0.00000000e+00  0.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [-1.86506793e-01 -5.59342623e-01  8.32816720e-01  1.02040808e+00
   0.00000000e+00  0.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [ 2.41135031e-01 -3.39085519e-01  2.85039127e-01  5.23308980e-01
   1.39570385e-01 -1.00000000e+00  9.28273773e-01  3.50389498e-31
   2.46285473e-08  7.17262022e-02]
 [ 3.92831042e-02 -5.33591770e-02  6.62491797e-03  6.65901913e-02
   0.00000000e+00  0.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [ 4.52355117e-01  8.363699


numEvents: 38802
num_FSPs_toData: 15
leaves.shape: (38802, 15, 10)
SA_target.shape: (38802, 15)
global_tag.shape: (38802, 16)
leaves[0]: [[ 2.62307338e-002  1.08511567e-001  1.62839703e-002  1.12818346e-001
   0.00000000e+000  0.00000000e+000 -1.00000000e+000 -1.00000000e+000
  -1.00000000e+000 -1.00000000e+000]
 [ 4.24084738e-002 -4.39727418e-002  1.09497361e-001  1.25386414e-001
   0.00000000e+000  0.00000000e+000 -1.00000000e+000 -1.00000000e+000
  -1.00000000e+000 -1.00000000e+000]
 [ 1.79167569e-001 -3.58085558e-002  1.23985238e-001  2.20806725e-001
   0.00000000e+000  0.00000000e+000 -1.00000000e+000 -1.00000000e+000
  -1.00000000e+000 -1.00000000e+000]
 [-8.97912011e-002 -5.98760974e-003  4.77474928e-002  1.01873129e-001
   0.00000000e+000  0.00000000e+000 -1.00000000e+000 -1.00000000e+000
  -1.00000000e+000 -1.00000000e+000]
 [-7.10087046e-002  1.10427499e-001  2.83671081e-001  3.42323887e-001
   1.39570385e-001  1.00000000e+000  4.51228108e-001  7.55338168e-012
   2.26945150e


numEvents: 22255
num_FSPs_toData: 18
leaves.shape: (22255, 18, 10)
SA_target.shape: (22255, 18)
global_tag.shape: (22255, 19)
leaves[0]: [[-9.52184498e-02  3.62810865e-02  1.20835707e-01  1.58063716e-01
   0.00000000e+00  0.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [-9.77953598e-02 -2.94023510e-02 -1.95226505e-01  2.20322080e-01
   0.00000000e+00  0.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [-7.29059130e-02 -1.50300771e-01  2.79857635e-01  3.25923135e-01
   0.00000000e+00  0.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [ 2.28249267e-01  1.30148426e-01  2.30528824e-02  2.98408559e-01
   1.39570385e-01 -1.00000000e+00  2.40694131e-01  1.43096956e-17
   7.27219731e-01  3.20861380e-02]
 [-5.09604029e-02  9.83461272e-03  1.18720429e-02  5.32412216e-02
   0.00000000e+00  0.00000000e+00 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [ 8.66104290e-02 -9.181203

In [ ]:
print("saving is done")
now = datetime.now()
print("time at end =", now)